In [1]:
seed=197

In [2]:
# importing required libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import random
import pickle
import torch.nn.init as init
from torchviz import make_dot
import plotly.graph_objects as go
import pickle
# import seaborn as sns
# import torchtext
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import display
import matplotlib.pyplot as plt
import os 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

warnings.simplefilter("ignore")
print(torch.__version__)

1.13.1+cu117


In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    # CUDA is available, so PyTorch can use a GPU
    device = torch.device("cuda")
    print("PyTorch is using GPU.")
else:
    # CUDA is not available, PyTorch will use CPU
    device = torch.device("cpu")
    print("PyTorch is using CPU.")

# Create a tensor and move it to the device
tensor = torch.randn(3, 4).to(device)

# Check the device of the tensor
print("Tensor device:", tensor.device)

PyTorch is using GPU.
Tensor device: cuda:0


In [4]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
#     tf.random.set_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [74]:
# pickle_file_path = '/media/terminator/Data/NatureData/Auto Ten Minutes.pkl'
pickle_file_path = '/media/terminator/NatureData/Ten Minutes.pkl'

In [75]:
# Load data from the pickle file
with open(pickle_file_path, "rb") as f:
    loaded_data_dict = pickle.load(f)


In [76]:
# Retrieve data and labels
X_train = loaded_data_dict["X_train"]
y_train = loaded_data_dict["y_train"]
X_test = loaded_data_dict["X_test"]
y_test = loaded_data_dict["y_test"]

In [77]:
X_train.shape,y_train.shape

(torch.Size([23328, 144, 13]), torch.Size([23328, 3]))

In [78]:
features=13

In [79]:
def reconstructData(packets):
    # Take the first packet
    first_packet = packets[0]
    
    # Concatenate the last index of every packet
    reconstructed_data = np.concatenate([packet[-1:] for packet in packets[1:]])
    
    # Combine the first packet with the concatenated data
    reconstructed_data = np.concatenate([first_packet, reconstructed_data])
    
    return reconstructed_data

In [100]:
Valid,Test = X_test[:100000],X_test[100000:]
Valid_label,Test_label = y_test[:100000],y_test[100000:]

In [101]:
num_epochs = 500
# Define batch size
batch_size = 256
# Create DataLoader for training set
train_dataset = torch.utils.data.TensorDataset(X_train.float(), y_train.float())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_dataset = torch.utils.data.TensorDataset(Valid.float(), Valid_label.float())
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(Test.float(), Test_label.float())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [102]:
print(X_train.dtype)
print(y_train.dtype)

torch.float64
torch.float64


In [103]:
X_train.shape, y_train.shape

(torch.Size([23328, 144, 13]), torch.Size([23328, 3]))

In [129]:
import torch
import torch.nn as nn

class TimeSeriesEncoder(nn.Module):
    def __init__(self, input_channels):
        super(TimeSeriesEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1),  # Reduce sequence length to 36 / 4 = 9
            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1)  # Reduce sequence length to 9 / 2 = 4.5 (rounded to 4)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*142, 1024),
            nn.ReLU(), 
            nn.Linear(1024, 3), 
            nn.Softmax(dim=1)  
        )
        
    
    def forward(self, x):
        x = x.permute(0,2,1)
        x = self.encoder(x)
#         print(x.shape)
        x = torch.reshape(x,(x.shape[0],-1))
        x = self.classifier(x)
#         x = x.permute(0,2,1)
        return x

# Create the autoencoder model
input_channels = 13  # Number of input features
# encoder = TimeSeriesEncoder(input_channels).to(device)





In [130]:
train = 1
avg = 'weighted'
seq_length = 144
num_epochs = 5000
reset_random_seeds(seed)
src_vocab_size = features+5
target_vocab_size = features
num_layers = 2
latent_dim = 8
embed_dim = 128
n_heads = 8
expansion_factor = 2

Cf = TimeSeriesEncoder(input_channels).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_cf = optim.Adam(Cf.parameters(), lr=0.00001)

# Define early stopping parameters
patience = 100
best_loss = 0
epochs_without_improvement = 0

if train:
    # Step 4: Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0
        latents = []
        PR, PL = [], []
        train_input,train_output = [], []

        # Iterate over mini-batches
        for i, (inputs, labels) in enumerate(train_loader):
            
            probability = 0.0
            coin_toss = random.random()  

            if coin_toss < probability:
                continue
            
            # Clear gradients
            optimizer_cf.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            
            prediction = Cf(inputs)
            
            
            # Calculate loss
            loss = criterion(prediction, labels)
            loss = loss.float()

            # Backpropagation
            loss.backward()
            
            optimizer_cf.step()
            
            # Update running loss
            running_loss += loss.item()
            
            
            PR.append(prediction)
            PL.append(labels)
            

        # Calculate average loss per epoch
        PR = torch.cat(PR, dim=0)
        PL = torch.cat(PL, dim=0)
        _, predicted = torch.max(PR.data, 1)
        _, lbl = torch.max(PL.data, 1)
        total = lbl.size(0)
        correct = (predicted == lbl).sum().item()

        accuracy = 100 * correct / total
        print(f'Train Accuracy: {accuracy:.2f}%')
        conf_matrix = confusion_matrix(lbl.cpu().detach().numpy(),predicted.cpu().detach().numpy())
        print('Confusion Matrix:')
        print(conf_matrix)
        # Calculate average loss per epoch
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch: {epoch+1}, Train Loss: {round(epoch_loss,6)}")
        
        if epoch%1==0:
            valid_latents = []
            VR,VL = [], []
            valid_loss = 0.0
            for j, (inp, lab) in enumerate(valid_loader):
                probability = 0.0
                coin_toss = random.random()  

                if coin_toss < probability:
                    continue
                inp = inp.to(device)
                lab = lab.to(device)
                
                with torch.no_grad():
                    # Forward pass
                    outp = Cf(inp)
                    
                VR.append(outp)
                VL.append(lab)
            
            VR = torch.cat(VR, dim=0)
            VL = torch.cat(VL, dim=0)
            _, vpre = torch.max(VR.data, 1)
            _, vlbl = torch.max(VL.data, 1)
            total = vlbl.size(0)
            correct = (vpre == vlbl).sum().item()

            vaccuracy = 100 * correct / total
            print('-----------------------------------------')
            print(f'Valid Accuracy: {vaccuracy:.2f}%')
            lb = vlbl.cpu().detach().numpy()
            pre = vpre.cpu().detach().numpy()
            conf_matrix = confusion_matrix(lb,pre)
            precision = precision_score(lb, pre,average=avg)
            recall = recall_score(lb, pre,average=avg)
            f1 = f1_score(lb, pre,average=avg)
            print('Confusion Matrix:')
            print(conf_matrix)
            print()
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1)
            print(epochs_without_improvement, best_loss)
            print('-----------------------------------------')
            print()
            print()
            
                
                    
        # Check for early stopping and save the best weights
        th = (conf_matrix[0][0]> conf_matrix[0][1]+conf_matrix[0][2] and conf_matrix[1][1]>conf_matrix[1][0]+conf_matrix[1][2] and conf_matrix[2][2]>conf_matrix[2][0]+conf_matrix[2][1])
        if th and vaccuracy > best_loss:
            best_loss = vaccuracy
            epochs_without_improvement = 0
            # Save the model weights
            torch.save(Cf.state_dict(), 'cnn10.pt')
            print('##############################################')
            print('saved', best_loss)
            print('##############################################')
            print()
            print()
            
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered. Training stopped.")
                break

# Recover the best weights
Cf.load_state_dict(torch.load('cnn10.pt'))


Train Accuracy: 48.23%
Confusion Matrix:
[[4159    0 3617]
 [2044    0 5732]
 [ 684    0 7092]]
Epoch: 1, Train Loss: 1.079045
-----------------------------------------
Valid Accuracy: 51.80%
Confusion Matrix:
[[51054     0 45428]
 [  665     0  2084]
 [   26     0   743]]

Precision: 0.9520542503837298
Recall: 0.51797
F1 Score: 0.6648612722841476
0 0
-----------------------------------------


Train Accuracy: 51.01%
Confusion Matrix:
[[4375    1 3400]
 [1875    2 5899]
 [ 253    0 7523]]
Epoch: 2, Train Loss: 1.022987
-----------------------------------------
Valid Accuracy: 52.24%
Confusion Matrix:
[[51500   114 44868]
 [  683     2  2064]
 [   26     0   743]]

Precision: 0.952311523273576
Recall: 0.52245
F1 Score: 0.6686164004181419
1 0
-----------------------------------------


Train Accuracy: 53.85%
Confusion Matrix:
[[4374  256 3146]
 [1863  667 5246]
 [ 254    0 7522]]
Epoch: 3, Train Loss: 0.987147
-----------------------------------------
Valid Accuracy: 52.96%
Confusion Mat

Train Accuracy: 74.43%
Confusion Matrix:
[[5302 2084  390]
 [2400 4769  607]
 [ 144  339 7293]]
Epoch: 18, Train Loss: 0.801649
-----------------------------------------
Valid Accuracy: 68.53%
Confusion Matrix:
[[66133 27367  2982]
 [  894  1707   148]
 [   23    51   695]]

Precision: 0.9546332149887353
Recall: 0.68535
F1 Score: 0.7856253797742774
0 67.561
-----------------------------------------


##############################################
saved 68.535
##############################################


Train Accuracy: 75.03%
Confusion Matrix:
[[5319 2079  378]
 [2316 4884  576]
 [ 144  331 7301]]
Epoch: 19, Train Loss: 0.797297
-----------------------------------------
Valid Accuracy: 68.93%
Confusion Matrix:
[[66523 27059  2900]
 [  891  1710   148]
 [   23    51   695]]

Precision: 0.9548023824666524
Recall: 0.68928
F1 Score: 0.7884500276971039
0 68.535
-----------------------------------------


##############################################
saved 68.928
#######################

Train Accuracy: 79.44%
Confusion Matrix:
[[5465 1994  317]
 [1585 5747  444]
 [ 283  174 7319]]
Epoch: 34, Train Loss: 0.763331
-----------------------------------------
Valid Accuracy: 70.88%
Confusion Matrix:
[[68229 25862  2391]
 [  681  1941   127]
 [   32    31   706]]

Precision: 0.9584427935019116
Recall: 0.70876
F1 Score: 0.802087239813304
0 70.884
-----------------------------------------


Train Accuracy: 79.58%
Confusion Matrix:
[[5476 1987  313]
 [1565 5768  443]
 [ 285  171 7320]]
Epoch: 35, Train Loss: 0.762106
-----------------------------------------
Valid Accuracy: 70.99%
Confusion Matrix:
[[68343 25803  2336]
 [  678  1944   127]
 [   32    34   703]]

Precision: 0.9585309384221041
Recall: 0.7099
F1 Score: 0.8029222742035186
1 70.884
-----------------------------------------


##############################################
saved 70.99
##############################################


Train Accuracy: 79.63%
Confusion Matrix:
[[5478 1986  312]
 [1558 5777  441]
 [ 285  1

Train Accuracy: 80.92%
Confusion Matrix:
[[5527 1976  273]
 [1375 6019  382]
 [ 292  154 7330]]
Epoch: 51, Train Loss: 0.748155
-----------------------------------------
Valid Accuracy: 71.78%
Confusion Matrix:
[[69105 25401  1976]
 [  620  2005   124]
 [   44    59   666]]

Precision: 0.9594961217810268
Recall: 0.71776
F1 Score: 0.8086331272856481
0 71.751
-----------------------------------------


##############################################
saved 71.776
##############################################


Train Accuracy: 80.99%
Confusion Matrix:
[[5529 1976  271]
 [1370 6035  371]
 [ 293  153 7330]]
Epoch: 52, Train Loss: 0.747469
-----------------------------------------
Valid Accuracy: 71.83%
Confusion Matrix:
[[69154 25364  1964]
 [  620  2005   124]
 [   44    52   673]]

Precision: 0.9595286360246456
Recall: 0.71832
F1 Score: 0.8090052825628353
0 71.776
-----------------------------------------


##############################################
saved 71.832
#######################

Train Accuracy: 81.61%
Confusion Matrix:
[[5643 1872  261]
 [1360 6066  350]
 [ 352   94 7330]]
Epoch: 67, Train Loss: 0.739211
-----------------------------------------
Valid Accuracy: 72.61%
Confusion Matrix:
[[69872 24615  1995]
 [  594  2033   122]
 [   51    14   704]]

Precision: 0.960010279625174
Recall: 0.72609
F1 Score: 0.8141734153436315
0 72.556
-----------------------------------------


##############################################
saved 72.609
##############################################


Train Accuracy: 81.66%
Confusion Matrix:
[[5647 1868  261]
 [1353 6073  350]
 [ 369   77 7330]]
Epoch: 68, Train Loss: 0.738744
-----------------------------------------
Valid Accuracy: 72.66%
Confusion Matrix:
[[69922 24565  1995]
 [  594  2033   122]
 [   51    14   704]]

Precision: 0.9600204708395123
Recall: 0.72659
F1 Score: 0.8145158015614913
0 72.609
-----------------------------------------


##############################################
saved 72.659
########################

...

-----------------------------------------
Valid Accuracy: 94.38%
Confusion Matrix:
[[91388  3925  1169]
 [  370  2313    66]
 [   90     3   676]]

Precision: 0.9728963588316722
Recall: 0.94377
F1 Score: 0.9543917085987809
35 94.909
-----------------------------------------


Train Accuracy: 92.23%
Confusion Matrix:
[[7565  122   89]
 [ 954 6615  207]
 [ 352   89 7335]]
Epoch: 1299, Train Loss: 0.628781
-----------------------------------------
Valid Accuracy: 94.33%
Confusion Matrix:
[[91335  3925  1222]
 [  370  2313    66]
 [   79     3   687]]

Precision: 0.9729632996546372
Recall: 0.94335
F1 Score: 0.9541379678231191
36 94.909
-----------------------------------------


Train Accuracy: 92.23%
Confusion Matrix:
[[7565  122   89]
 [ 953 6615  208]
 [ 354   87 7335]]
Epoch: 1300, Train Loss: 0.628932
-----------------------------------------
Valid Accuracy: 94.04%
Confusion Matrix:
[[91037  4234  1211]
 [  370  2313    66]
 [   79     3   687]]

Precision: 0.9724822747896376
Recall: 

Train Accuracy: 92.23%
Confusion Matrix:
[[7566  122   88]
 [ 950 6615  211]
 [ 332  109 7335]]
Epoch: 1319, Train Loss: 0.62877
-----------------------------------------
Valid Accuracy: 94.39%
Confusion Matrix:
[[91387  3899  1196]
 [  370  2313    66]
 [   79     4   686]]

Precision: 0.9730400759156096
Recall: 0.94386
F1 Score: 0.9544843523095656
56 94.909
-----------------------------------------


Train Accuracy: 92.23%
Confusion Matrix:
[[7566  122   88]
 [ 948 6615  213]
 [ 335  106 7335]]
Epoch: 1320, Train Loss: 0.628748
-----------------------------------------
Valid Accuracy: 94.14%
Confusion Matrix:
[[91139  4158  1185]
 [  370  2313    66]
 [   79     4   686]]

Precision: 0.9726335159642644
Recall: 0.94138
F1 Score: 0.9527921149321931
57 94.909
-----------------------------------------


Train Accuracy: 92.23%
Confusion Matrix:
[[7566  122   88]
 [ 954 6615  207]
 [ 334  107 7335]]
Epoch: 1321, Train Loss: 0.628727
-----------------------------------------
Valid Accuracy:

-----------------------------------------
Valid Accuracy: 94.12%
Confusion Matrix:
[[91114  4190  1178]
 [  370  2313    66]
 [   73     3   693]]

Precision: 0.9726761418103993
Recall: 0.9412
F1 Score: 0.9526833346520619
76 94.909
-----------------------------------------


Train Accuracy: 92.24%
Confusion Matrix:
[[7568  121   87]
 [ 951 6615  210]
 [ 352   89 7335]]
Epoch: 1340, Train Loss: 0.62873
-----------------------------------------
Valid Accuracy: 94.21%
Confusion Matrix:
[[91224  4094  1164]
 [  370  2313    66]
 [   96     3   670]]

Precision: 0.9725477494833918
Recall: 0.94207
F1 Score: 0.9532165740377205
77 94.909
-----------------------------------------


Train Accuracy: 92.24%
Confusion Matrix:
[[7568  121   87]
 [ 954 6615  207]
 [ 339  102 7335]]
Epoch: 1341, Train Loss: 0.628667
-----------------------------------------
Valid Accuracy: 94.38%
Confusion Matrix:
[[91368  3919  1195]
 [  370  2313    66]
 [   72     0   697]]

Precision: 0.9731154113389345
Recall: 0.

Train Accuracy: 92.25%
Confusion Matrix:
[[7569  121   86]
 [ 951 6615  210]
 [ 340  101 7335]]
Epoch: 1360, Train Loss: 0.628648
-----------------------------------------
Valid Accuracy: 93.96%
Confusion Matrix:
[[90989  4355  1138]
 [  370  2313    66]
 [  104     7   658]]

Precision: 0.9720631612764529
Recall: 0.9396
F1 Score: 0.9515288653008784
97 94.909
-----------------------------------------


Train Accuracy: 92.25%
Confusion Matrix:
[[7569  121   86]
 [ 952 6615  209]
 [ 328  113 7335]]
Epoch: 1361, Train Loss: 0.628837
-----------------------------------------
Valid Accuracy: 93.96%
Confusion Matrix:
[[90940  4325  1217]
 [  370  2313    66]
 [   64     2   703]]

Precision: 0.9725354337788019
Recall: 0.93956
F1 Score: 0.9515965226879721
98 94.909
-----------------------------------------


Train Accuracy: 92.25%
Confusion Matrix:
[[7569  121   86]
 [ 943 6615  218]
 [ 338  103 7335]]
Epoch: 1362, Train Loss: 0.628731
-----------------------------------------
Valid Accuracy:

<All keys matched successfully>

In [131]:
Cf.load_state_dict(torch.load('cnn10.pt'))

<All keys matched successfully>

In [132]:
valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(train_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Train Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(valid_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Valid Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(test_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Test Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

-----------------------------------------
Train Accuracy: 92.23%
Confusion Matrix:
[[7565  122   89]
 [1000 6615  161]
 [ 441    0 7335]]

Precision: 0.9296422729625746
Recall: 0.9222822359396433
F1 Score: 0.9227245485641565
100 94.909
-----------------------------------------


-----------------------------------------
Valid Accuracy: 94.91%
Confusion Matrix:
[[91971  3359  1152]
 [  370  2313    66]
 [  141     3   625]]

Precision: 0.9733011187691218
Recall: 0.94909
F1 Score: 0.9579546046267547
100 94.909
-----------------------------------------


-----------------------------------------
Test Accuracy: 95.84%
Confusion Matrix:
[[153556   4537   1506]
 [   541   2913     57]
 [   177      7    788]]

Precision: 0.9784957801373904
Recall: 0.9584049438695287
F1 Score: 0.9659019124394538
100 94.909
-----------------------------------------


